# Refresh FAOSTAT data in Datamart

In [1]:
# Parameters to be injected
template_path = 'FAOSTAT_template.tsv'
datasets_path = './datasets'
datamart_api_url = 'http://localhost:12543'

In [2]:
import io
import os.path
import pandas as pd
import json
from requests import get,post,put,delete
from IPython.display import display, HTML

In [23]:
def upload_frame_annotated(buffer, url, put_data=True):
    
    buffer.seek(0)
    
    files = {
        'file': ('buffer.csv', buffer, 'application/octet-stream')
    }
    
    if put_data:
        response = put(url, files=files)
    else:
        response = post(url, files=files)
    if response.status_code == 400:
        print(json.dumps(response.json(), indent=2))
    else:
        print(json.dumps(response.json(), indent=2))

In [24]:
if not os.path.isfile(template_path):
    raise FileNotFoundError("Template file does not exist!")
    
df_template = pd.read_csv(template_path, sep='\t', dtype=object, header=None).fillna('')
df_template

,0,1,2,3,4,5,6,7,8,9,10,11
0,dataset,FAOSTAT,FAO Fertilizers by Product,The Fertilizers by Product dataset contains in...,http://www.fao.org/faostat/en/#data/RFB,,,,,,,
1,role,qualifier,location,qualifier,main subject,qualifier,qualifier,,time,unit,variable,qualifier
2,type,string,country,string,string,number,string,,year,string,number,string
3,description,,,,,,,,,,,
4,name,,,,,,,,,,,
5,unit,,,,,,,,,,,
6,header,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag


In [25]:
def generate_filenames(datasets_path: str):
    files_to_upload = []
    if os.path.isfile(datasets_path):
        files_to_upload.append(datasets_path)
    else:
        files_to_upload += [ os.path.join(datasets_path, f) for f in os.listdir(datasets_path) if os.path.isfile(os.path.join(datasets_path,f)) ]
    return files_to_upload

files_to_upload = generate_filenames(datasets_path)

In [27]:
dataset_id = df_template.iloc[0,1]
nCols = len(df_template.iloc[0]) - 1
for i, data_path in enumerate(files_to_upload):
        
    print(data_path)

    # Only extract the first nCols columns specified in the template
    sheet = pd.DataFrame()
    if data_path.endswith('.xlsx'):
        sheet = pd.read_excel(data_path, dtype=object).fillna('')
    elif data_path.endswith('.csv'):
        sheet = pd.read_csv(data_path, encoding='latin1', dtype=object).fillna('')
    else:
        continue
        
    sheet = sheet[sheet.columns[:nCols]]
    
    # Verify Label matches
    if False in sheet.columns == df_template.iloc[6][1:]:
        raise ValueError(f'Columns do not match between template and input: {data_path}. Abort...')
        
    # Build inputs
    sheet.insert(loc=0, column='', value='')
    sheet.iloc[0,0] = 'data'
            
    # Build annotated data
    sheet.columns = df_template.columns
    annotated_sheet = df_template.append(sheet)
        
    # post data to datamart
    buffer = io.StringIO()
    annotated_sheet.to_csv(buffer, index=False, header=False)
    url = f'{datamart_api_url}/datasets/{dataset_id}/annotated?create_if_not_exist=true'
    upload_frame_annotated(buffer, url, False)
    #annotated_sheet.to_excel('tmp.xlsx', index=False, header=False)

./datasets\Inputs_FertilizersProduct_E_All_Data_(Normalized).csv
[
  {
    "name": "Value",
    "variable_id": "value",
    "dataset_id": "FAOSTAT",
    "description": "variable column in QFAOSTAT",
    "corresponds_to_property": "PVARIABLE-QFAOSTAT-007",
    "qualifier": [
      {
        "name": "Flag",
        "identifier": "PQUALIFIER-QFAOSTAT-008",
        "data_type": "String"
      },
      {
        "name": "Element",
        "identifier": "PQUALIFIER-QFAOSTAT-006",
        "data_type": "String"
      },
      {
        "name": "Element Code",
        "identifier": "PQUALIFIER-QFAOSTAT-005",
        "data_type": "Quantity"
      },
      {
        "name": "located in the administrative territorial entity",
        "identifier": "P131"
      },
      {
        "name": "Item Code",
        "identifier": "PQUALIFIER-QFAOSTAT-003",
        "data_type": "String"
      },
      {
        "name": "Area Code",
        "identifier": "PQUALIFIER-QFAOSTAT-002",
        "data_type": "Strin

In [3]:
dataset_id = 'FAOSTAT'
q_variable = 'value'
response = get(f'{datamart_api_url}/datasets/{dataset_id}/variables/{q_variable}')
df = pd.read_csv(io.StringIO(response.text)).head(20)
display(HTML(df.fillna('').to_html(index=False)))

dataset_id,variable_id,variable,main_subject,main_subject_id,value,value_unit,time,time_precision,country,admin1,admin2,admin3,region_coordinate,stated_in,stated_in_id,Flag,Element,Element Code,located in the administrative territorial entity,Item Code,Area Code,stated in
FAOSTAT,value,Value,"Item Ammonia, anhydrous","QQFAOSTAT_Item_Ammonia,_anhydrous",146.00,tonnes,2002-01-01T00:00:00Z,year,,,,,,,,R,Import Quantity,5610.0,United Arab Emirates,4007,225,
FAOSTAT,value,Value,"Item Ammonia, anhydrous","QQFAOSTAT_Item_Ammonia,_anhydrous",619632.40,1000 US$,2002-01-01T00:00:00Z,year,,,,,,,,R,Import Value,5622.0,United States of America,4007,231,
FAOSTAT,value,Value,"Item Ammonia, anhydrous","QQFAOSTAT_Item_Ammonia,_anhydrous",41749.01,tonnes,2002-01-01T00:00:00Z,year,,,,,,,,X,Import Quantity,5610.0,Vietnam,4007,237,
FAOSTAT,value,Value,"Item Ammonia, anhydrous","QQFAOSTAT_Item_Ammonia,_anhydrous",5924491.24,tonnes,2002-01-01T00:00:00Z,year,,,,,,,,R,Import Quantity,5610.0,United States of America,4007,231,
FAOSTAT,value,Value,"Item Ammonia, anhydrous","QQFAOSTAT_Item_Ammonia,_anhydrous",14.40,1000 US$,2002-01-01T00:00:00Z,year,,,,,,,,Qm,Export Value,5922.0,Zimbabwe,4007,181,
FAOSTAT,value,Value,"Item Ammonia, anhydrous","QQFAOSTAT_Item_Ammonia,_anhydrous",48.63,1000 US$,2002-01-01T00:00:00Z,year,,,,,,,,Qm,Import Value,5622.0,Uruguay,4007,234,
FAOSTAT,value,Value,"Item Ammonia, anhydrous","QQFAOSTAT_Item_Ammonia,_anhydrous",70.31,1000 US$,2002-01-01T00:00:00Z,year,,,,,,,,Qm,Import Value,5622.0,Tanzania,4007,215,
FAOSTAT,value,Value,"Item Ammonia, anhydrous","QQFAOSTAT_Item_Ammonia,_anhydrous",4723.90,tonnes,2002-01-01T00:00:00Z,year,,,,,,,,R,Export Quantity,5910.0,Uzbekistan,4007,235,
FAOSTAT,value,Value,"Item Ammonia, anhydrous","QQFAOSTAT_Item_Ammonia,_anhydrous",24.83,1000 US$,2002-01-01T00:00:00Z,year,,,,,,,,Qm,Import Value,5622.0,Zambia,4007,251,
FAOSTAT,value,Value,"Item Ammonia, anhydrous","QQFAOSTAT_Item_Ammonia,_anhydrous",6064.16,1000 US$,2002-01-01T00:00:00Z,year,,,,,,,,Qm,Import Value,5622.0,Vietnam,4007,237,
